In [16]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

In [17]:
from api_keys import g_key

In [18]:
output_data_file = "Weather_Cities.csv"
cities_wth_2 = pd.read_csv(output_data_file)
cities_wth_2

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Ushuaia,20,AR,1570079116,97,-54.81,-68.31,41.00,26.40
1,Provideniya,75,RU,1570079116,86,64.42,-173.23,37.40,15.66
2,Chuy,80,UY,1570078949,78,-33.69,-53.46,56.29,11.86
3,Bluff,0,AU,1570079117,23,-23.58,149.07,82.89,15.48
4,Joshimath,0,IN,1570079117,52,30.57,79.57,61.56,6.49
...,...,...,...,...,...,...,...,...,...
541,Palu,32,ID,1570079404,83,-0.90,119.87,80.78,3.22
542,Te Anau,20,NZ,1570079404,85,-45.41,167.72,43.04,10.04
543,Lujiang,40,CN,1570079404,48,31.26,117.28,86.00,4.47
544,Umm Lajj,0,SA,1570079404,77,25.02,37.27,83.39,11.63


In [19]:
gmaps.configure(api_key=g_key)

In [20]:
locations = cities_wth_2[["Lat", "Lng"]]
humidity = cities_wth_2["Humidity"].astype(float)

In [21]:
fig = gmaps.figure(center=(20,0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=600)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
ideal_wth = cities_wth_2.loc[(cities_wth_2["Max Temp"] < 80) & (cities_wth_2["Max Temp"] > 70) &
                             (cities_wth_2["Wind Speed"] < 10) & (cities_wth_2["Cloudiness"] == 0)]
ideal_wth = pd.DataFrame(ideal_wth)
ideal_wth.dropna()
ideal_wth

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
28,Georgetown,0,GY,1570079130,94,6.80,-58.16,77.00,2.64
94,Mersin,0,TR,1570079169,88,36.81,34.63,75.20,5.82
162,Vila Velha,0,BR,1570079194,73,-3.71,-38.60,77.00,5.82
185,Ravar,0,IR,1570079211,16,31.27,56.81,75.65,7.02
270,Shenjiamen,0,CN,1570079242,79,29.96,122.30,78.05,9.51
302,Vieux-Habitants,0,GP,1570079261,88,16.06,-61.77,75.20,2.24
360,Ipixuna,0,BR,1570079301,83,-1.76,-48.80,77.00,8.05
398,Carutapera,0,BR,1570079314,77,-1.20,-46.02,78.16,6.78
458,Buraydah,0,SA,1570079347,41,26.33,43.97,78.80,4.70
484,Accra,0,GH,1570079223,94,5.56,-0.21,75.20,8.05


In [23]:
ideal_wth.count()


City          11
Cloudiness    11
Country       11
Date          11
Humidity      11
Lat           11
Lng           11
Max Temp      11
Wind Speed    11
dtype: int64

In [24]:
ideal_wth['Hotel Name'] = ""
ideal_wth

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
28,Georgetown,0,GY,1570079130,94,6.80,-58.16,77.00,2.64,
94,Mersin,0,TR,1570079169,88,36.81,34.63,75.20,5.82,
162,Vila Velha,0,BR,1570079194,73,-3.71,-38.60,77.00,5.82,
185,Ravar,0,IR,1570079211,16,31.27,56.81,75.65,7.02,
270,Shenjiamen,0,CN,1570079242,79,29.96,122.30,78.05,9.51,
302,Vieux-Habitants,0,GP,1570079261,88,16.06,-61.77,75.20,2.24,
360,Ipixuna,0,BR,1570079301,83,-1.76,-48.80,77.00,8.05,
398,Carutapera,0,BR,1570079314,77,-1.20,-46.02,78.16,6.78,
458,Buraydah,0,SA,1570079347,41,26.33,43.97,78.80,4.70,
484,Accra,0,GH,1570079223,94,5.56,-0.21,75.20,8.05,


In [25]:
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in ideal_wth.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_hotel = requests.get(base_url, params=params)
    

    # convert to json
    name_hotel = name_hotel.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        ideal_wth.loc[index, "Hotel Name"] = name_hotel["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [26]:
hotel_df = pd.DataFrame(ideal_wth)
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
28,Georgetown,0,GY,1570079130,94,6.80,-58.16,77.00,2.64,Georgetown
94,Mersin,0,TR,1570079169,88,36.81,34.63,75.20,5.82,Mersin
162,Vila Velha,0,BR,1570079194,73,-3.71,-38.60,77.00,5.82,Fortaleza
185,Ravar,0,IR,1570079211,16,31.27,56.81,75.65,7.02,Ravar
270,Shenjiamen,0,CN,1570079242,79,29.96,122.30,78.05,9.51,Zhoushan
302,Vieux-Habitants,0,GP,1570079261,88,16.06,-61.77,75.20,2.24,Baillif
360,Ipixuna,0,BR,1570079301,83,-1.76,-48.80,77.00,8.05,Colônia Velha
398,Carutapera,0,BR,1570079314,77,-1.20,-46.02,78.16,6.78,Carutapera
458,Buraydah,0,SA,1570079347,41,26.33,43.97,78.80,4.70,Buraydah
484,Accra,0,GH,1570079223,94,5.56,-0.21,75.20,8.05,Accra


In [27]:
gmaps.configure(api_key=g_key)

locations = hotel_df[["Lat", "Lng"]]
humidity = hotel_df["Humidity"].astype(float)

fig = gmaps.figure(center=(20,0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=50)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row)for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

hotel_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)


fig = gmaps.figure(center=(20,0), zoom_level=2)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
fig = gmaps.figure(center=(20,0), zoom_level=2)

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))